# Preprocessing: SLCH018


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'SLCH018'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/SLCH018/ieeg/SLCH018_notched_filtered_clean_ieeg.fif...


    Range : 0 ... 2779999 =      0.000 ...  1389.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH018/ieeg/SLCH018_notched_filtered_clean_ieeg-1.fif...
    Range : 2780000 ... 2958799 =   1390.000 ...  1479.399 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH018/ieeg/SLCH018_raw_clean_ieeg.fif...
    Range : 0 ... 2779999 =      0.000 ...  1389.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH018/ieeg/SLCH018_raw_clean_ieeg-1.fif...
    Range : 2780000 ... 2958799 =   1390.000 ...  1479.399 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [8]:
more_bads = [ 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35',
 'C36',
 'C37',
 'C38',
 'C39',
 'C40',
 'C41',
 'C42',
 'C43',
 'A61',
 'A62',
 'A63',
 'A64',
 'B61',
 'B62',
 'B63',
 'B64']

filtered_clean_fif.info['bads'].extend(more_bads)
raw_clean_fif.info['bads'].extend(more_bads)


In [10]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

NA noref
NA_2 noref
NA_3 noref
NA_4 noref
REF1 noref
`A1 `A2
`A2 `A3
`A3 `A4
`A4 `A5
`A5 `A6
`A6 `A7
`A7 `A8
`A8 `A9
`A9 `A10
`A10 `A11
`A11 `A12
`B1 `B2
`B2 `B3
`B3 `B4
`B4 `B5
`B5 `B6
`B6 `B7
`B7 `B8
`B8 `B9
`B9 `B10
`B10 `B11
`B11 `B12
`C1 noref
`C2 noref
`C3 noref
`C4 `C5
`C5 `C6
`C6 `C7
`C7 `C8
`C8 `C9
`C9 `C10
`C10 `C11
`C11 `C12
`C12 `C13
`C14 noref
`D1 `D2
`D2 `D3
`D3 `D4
`D4 `D5
`D5 `D6
`E1 `E2
`E2 `E3
`E3 `E4
`E4 `E5
`E5 `E6
`E6 `E7
`E7 `E8
`E8 `E9
`E10 noref
A61 noref
A62 noref
A63 noref
A64 noref
`F1 `F2
`F2 `F3
`F3 `F6
`F4 noref
`F5 noref
`F6 `F7
`F7 `F8
`F8 `F9
`F9 `F10
`G1 `G2
`G2 `G3
`G3 `G4
`G4 `G5
`G5 `G6
`G6 `G7
`G7 `G8
`G8 `G9
`G9 `G10
`G10 `G11
`G11 `G12
`G12 `G13
`G13 `G14
`G14 `G15
`G15 `G16
`H1 `H2
`H2 `H3
`H3 `H4
`H4 `H5
`H5 `H6
`H6 `H7
`H7 `H8
`i1 noref
`i2 noref
`i3 `i4
`i4 `i5
`i5 `i6
`i6 `i7
`i7 `i8
`i8 `i9
`i9 `i10
`i10 `i11
`i11 `i12
`i12 `i13
`i13 `i14
J`1 J`2
J`2 J`3
J`3 J`4
J`4 J`5
J`5 J`6
J`6 J`7
J`7 J`8
J`8 J`9
J`9 J`10
J`10 J`11
J`11 J`12
B61 noref


In [11]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 2958799  =      0.000 ...  1479.399 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=107, n_times=2958800
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.
Added the following bipolar channels:
`A1-`A2, `A2-`A3, `A3-`A4, `A4-`A5, `A5-`A6, `A6-`A7, `A7-`A8, `A8-`A9, `A9-`A10, `A10-`A11, `A11-`A12, `B1-`B2, `B2-`B3, `B3-`B4, `B4-`B5, `B5-`B6, `B6-`B7, `B7-`B8, `B8-`B9, `B9-`B10, `B10-`B11, `B11-`B12, `C4-`C5, `C5-`C6, `C6-`C7, `C7-`C8, `C8-`C9, `C9-`C10, `C10-`C11, `C11-`C12, `C12-`C13, `D1-`D2, `D2-`D3, `D3-`D4, `D4-`D5, `D5-`D6, `E1-`E2, `E2-`E3, `E3-`E4, `E4-`E5, `E5-`E6, `E6-`E7, `E7-`E8, `E8-`E9, `F1-`F2, `F2-`F3, `F3-`F6, `F6-`F7, `F7-`F8, `F8-`F9, `F9-`F10, `G1-`G2, `G2-`G3, `G3-`G4, `G4-`G5, `G5-`G6, `G6-`G7, `G7-`G8, `G8-`G9, `G9-`G10, `G10-`G11, `G11-`G12, `G12-`G13, `G13-`G14, `G14-`G15, `G15-`G16, `H1-`H2, `H2-`H3, `H3-`H4, `H4-`H5, `H5-`H6, `H6-`H7, `H7-`H8, `i3-`i4, `i4-`i5, `i5-`i6, `i6-`i7, 

/tmp/ipykernel_50731/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif
[done]
Reading 0 ... 2958799  =      0.000 ...  1479.399 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=107, n_times=2958800
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.
Added the following bipolar channels:
`A1-`A2, `A2-`A3, `A3-`A4, `A4-`A5, `A5-`A6, `A6-`A7, `A7-`A8, `A8-`A9, `A9-`A10, `A10-`A11, `A11-`A12, `B1-`B2, `B2-`B3, `B3-`B4, `B4-`B5, `B5-`B6, `B6-`B7, `B7-`B8, `B8-`B9, `B9-`B10, `B10-`B11, `B11-`B12, `C4-`C5, `C5-`C6, `C6-`C7, `C7-`C8, `C8-`C9, `C9-`C10, `C10-`C11, `C11-`C12, `C12-`C13, `D1-`D2, `D2-`D3, `D3-`D4, `D4-`D5, `D5-`D6, `E1-`E2, `E2-`E3, `E3-`E4, `E4-`E5, `E5-`E6, `E6-`E7, `E7-`E8, `E8-`E9, `F1-`F2, `F2-`F3, `F3-`F6, `F6-`F7, `F7-`F8, `F8-`F9, `F9-`F10, `G1-`G2, `G2-`G3, `G3-`G4, `G4-`G5, `G5-`G6, `G6-`G7, `G7-`G8, `G8-`G9, `G9-`G10, `G10-`G11, `G11-`G12, `G12-`G13, `G13-`G14, `G14-`G15, `G15-`G16, `H

/tmp/ipykernel_50731/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_raw_clean_data.fif
[done]


In [ ]:
# Visualize it #

# bp_raw_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

### Onset

In [12]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif...
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.


/tmp/ipykernel_50731/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 32001 original time points ...
Splitting into 2 parts
Overwriting existing file.
Loading data for 120 events and 32001 original time points ...


/tmp/ipykernel_50731/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Overwriting existing file.
Loading data for 120 events and 32001 original time points ...


### Trial End

In [13]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1] # remove last event
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif...
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.


/tmp/ipykernel_50731/3432568909.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Overwriting existing file.
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_50731/3432568909.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Overwriting existing file.
Loading data for 119 events and 20001 original time points ...


### Last Away

In [14]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,11,82500,0,1
1,1,18,102800,0,1
2,2,1,118400,0,1
3,3,2,127600,0,1
4,4,15,138700,0,1
...,...,...,...,...,...
223,233,3,2568700,0,1
224,234,4,2578400,0,1
225,235,13,2589400,0,1
226,237,7,2611100,0,1


In [15]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [16]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif...
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.
Not setting metadata
228 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Overwriting existing file.
Loading data for 228 events and 20001 original time points ...


/tmp/ipykernel_50731/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_50731/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [17]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,11,82000,0,1
1,1,18,89500,0,1
2,2,1,113600,0,1
3,3,2,126100,0,1
4,4,15,137100,0,1
...,...,...,...,...,...
229,234,4,2576000,0,1
230,235,13,2587700,0,1
231,236,8,2598100,0,1
232,237,7,2609400,0,1


In [18]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [19]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif...
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.
Not setting metadata
234 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 18001 original time points ...
Overwriting existing file.
Loading data for 234 events and 18001 original time points ...


/tmp/ipykernel_50731/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_50731/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


## First Move

In [20]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,11,81500,0,1
1,1,18,89300,0,1
2,2,1,113300,0,1
3,3,2,124300,0,1
4,4,15,136500,0,1
...,...,...,...,...,...
234,234,4,2575900,0,1
235,235,13,2585900,0,1
236,236,8,2597400,0,1
237,237,7,2609100,0,1


In [21]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [22]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif...
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Overwriting existing file.


/tmp/ipykernel_50731/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_50731/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...
Overwriting existing file.
Loading data for 119 events and 20001 original time points ...


## Ghost Attack

In [23]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,3,2,126800,0,1
1,5,4,148100,0,1
2,6,5,159700,0,1
3,12,12,211100,0,1
4,17,8,271100,0,1
5,23,3,392000,0,1
6,34,2,494000,0,1
7,36,12,515200,0,1
8,40,6,569600,0,1
9,48,4,646900,0,1


In [24]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [25]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif...
    Range : 0 ... 2958799 =      0.000 ...  1479.399 secs
Ready.
Not setting metadata
41 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Overwriting existing file.
Loading data for 41 events and 20001 original time points ...


/tmp/ipykernel_50731/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_50731/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH018/ieeg/SLCH018_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
